In [1]:
Special = ['sp', 'spn']

In [2]:
import tgt

tg = tgt.io.read_textgrid('03052019_step1.TextGrid.utf8')
ophonetier = tg.get_tier_by_name('IU/phone')
owordtier =  tg.get_tier_by_name('Word')

phonetier = ophonetier.get_copy_with_gaps_filled()
wordtier  = owordtier.get_copy_with_gaps_filled()

pst = phonetier.start_time
pet = phonetier.end_time

wst = wordtier.start_time
wet = wordtier.end_time

st = pst
et = pet

#print(wordtier)
#print(phonetier)

#ann = wordtier.get_annotations_between_timepoints(3.4203639873412612, 3.510363987341293)
#print(ann)

#ann = phonetier.get_annotations_between_timepoints(3.4203639873412612, 3.510363987341293, True, False)
#print(ann)

In [3]:
import numpy as np
import math

def remove_noise(tr):
    for o in tr._objects:
        if o.duration() < 0.01:
            tr.delete_annotation_by_start_time(o.start_time)

def filling_gap(tr):
    tr.start_time = 0
    o = tr._objects[0]
    if o.start_time != 0:
        o.start_time = 0
    o = tr._objects[-1]
    if o.end_time != tr.end_time:
        o.end_time = tr.end_time
    for i in range(1..len(tr._objects)):
        if tr._objects[i].start_time != tr._objects[i-1].end_time:
            tr._objects[i].start_time = tr.objects[i-1].end_time

def align_tiers(wtr, ptr):
    for w in wtr._objects:
        wst = w.start_time
        wet = w.end_time
        pann = ptr.get_annotations_between_timepoints(w.start_time, w.end_time, False, False)
        #print(wst, wet, pann[0].start_time, pann[-1].end_time)
        
        #print("Wst:", math.isclose(wst, pann[0].start_time), wst, pann[0].start_time)
        if not math.isclose(wst, pann[0].start_time):
            pann2 = ptr.get_annotations_between_timepoints(w.start_time, w.end_time, True, False)
            pst1 = pann[0].start_time
            pst2 = pann2[0].start_time
            #print("S:", w.start_time, pann[0].start_time, pann2[0].start_time)
            if abs(wst - pst1) < abs(wst - pst2):
                pann[0].start_time = wst
            else:
                pann2[0].start_time = wst
        
        #print("Wet:", math.isclose(wet, pann[-1].end_time), wet, pann[-1].end_time)
        if  not math.isclose(wet, pann[-1].end_time):
            pann2 = ptr.get_annotations_between_timepoints(w.start_time, w.end_time, False, True)
            pet1 = pann[-1].end_time
            pet2 = pann2[-1].end_time
            #print("E: ", wet, pann[-1].end_time, pann2[-1].end_time)
            if abs(wet - pet1) < abs(wet - pet2):
                pann[-1].end_time = wet
            else:
                pann2[-1].end_time = wet
            

In [4]:
nphonetier = ophonetier.get_copy_with_gaps_filled()
nwordtier  = owordtier.get_copy_with_gaps_filled()
remove_noise(nphonetier)
align_tiers(nwordtier, nphonetier)
print()
print(nphonetier)


IntervalTier(start_time=0.0, end_time=3.680363987341252, name="IU/phone", objects=[Interval(0.0, 0.03036398734127488, "n"), Interval(0.03036398734127488, 0.06036398734130444, "i"), Interval(0.06036398734130444, 0.2203639873412726, ""), Interval(0.2203639873412726, 0.25036398734130216, "Z"), Interval(0.25036398734130216, 0.34036398734127715, "%"), Interval(0.34036398734127715, 0.42036398734126124, "d"), Interval(0.42036398734126124, 0.4803639873412635, "a"), Interval(0.4803639873412635, 0.520363987341284, "w"), Interval(0.520363987341284, 0.5903639873412772, "g"), Interval(0.5903639873412772, 0.6303639873412976, "&"), Interval(0.6303639873412976, 0.6901303533302894, "n"), Interval(0.6901303533302894, 0.7247191411732956, "U"), Interval(0.7247191411732956, 0.7435857527240728, "y"), Interval(0.7435857527240728, 0.7687412347917189, "E"), Interval(0.7687412347917189, 0.7928485717732201, "n"), Interval(0.7928485717732201, 0.8169559087547782, "x"), Interval(0.8169559087547782, 0.8337262301332

In [5]:
newtg = tgt.core.TextGrid()

newtg.add_tier(nwordtier)
newtg.add_tier(nphonetier)
tgt.io.write_to_file(newtg, "newresult.TextGrid")
